Import Libraries

In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats as stats
import datetime


Load the datasets

In [4]:
df_customers = pd.read_csv('/Users/eranola/Documents/projects/sales_analysis/data/raw/olist_customers_dataset.csv')
df_geolocation = pd.read_csv('/Users/eranola/Documents/projects/sales_analysis/data/raw/olist_geolocation_dataset.csv')
df_order_items = pd.read_csv('/Users/eranola/Documents/projects/sales_analysis/data/raw/olist_order_items_dataset.csv')
df_order_payments = pd.read_csv('/Users/eranola/Documents/projects/sales_analysis/data/raw/olist_order_payments_dataset.csv')
df_order_reviews = pd.read_csv('/Users/eranola/Documents/projects/sales_analysis/data/raw/olist_order_reviews_dataset.csv')
df_orders = pd.read_csv('/Users/eranola/Documents/projects/sales_analysis/data/raw/olist_orders_dataset.csv')
df_products = pd.read_csv('/Users/eranola/Documents/projects/sales_analysis/data/raw/olist_products_dataset.csv')
df_sellers = pd.read_csv('/Users/eranola/Documents/projects/sales_analysis/data/raw/olist_sellers_dataset.csv')
df_product_category_translation = pd.read_csv('/Users/eranola/Documents/projects/sales_analysis/data/raw/product_category_name_translation.csv')

In [5]:
# Convert relevant columns in the df_orders dataset to datetime
df_orders['order_purchase_timestamp'] = pd.to_datetime(df_orders['order_purchase_timestamp'])
df_orders['order_approved_at'] = pd.to_datetime(df_orders['order_approved_at'])
df_orders['order_delivered_carrier_date'] = pd.to_datetime(df_orders['order_delivered_carrier_date'])
df_orders['order_delivered_customer_date'] = pd.to_datetime(df_orders['order_delivered_customer_date'])
df_orders['order_estimated_delivery_date'] = pd.to_datetime(df_orders['order_estimated_delivery_date'])

# Convert relevant columns in the df_order_reviews dataset to datetime
df_order_reviews['review_creation_date'] = pd.to_datetime(df_order_reviews['review_creation_date'])
df_order_reviews['review_answer_timestamp'] = pd.to_datetime(df_order_reviews['review_answer_timestamp'])

# Convert relevant columns in the df_order_items dataset to datetime
df_order_items['shipping_limit_date'] = pd.to_datetime(df_order_items['shipping_limit_date'])


In [9]:
# Merge orders with order payments to get the complete picture
df_order_details = pd.merge(df_orders, df_order_payments, on='order_id')

# Convert timestamps to datetime
df_order_details['order_purchase_timestamp'] = pd.to_datetime(df_order_details['order_purchase_timestamp'])

# Extract year-month from the order purchase date
df_order_details['year_month'] = df_order_details['order_purchase_timestamp'].dt.to_period('M')

# Calculate total revenue per order
df_order_details['total_revenue'] = df_order_details['payment_value']

# 1. Revenue per Customer (Monthly)
monthly_revenue_per_customer = df_order_details.groupby(['year_month', 'customer_id'])['total_revenue'].sum().reset_index()
monthly_revenue_per_customer_avg = monthly_revenue_per_customer.groupby('year_month')['total_revenue'].mean()

# 2. Average Transaction Value (Monthly)
# Average Transaction Value = Total Revenue / Number of Orders
monthly_total_revenue = df_order_details.groupby('year_month')['total_revenue'].sum()
monthly_total_orders = df_order_details.groupby('year_month')['order_id'].nunique()
average_transaction_value = monthly_total_revenue / monthly_total_orders

# 3. Revenue Growth Rate (Monthly)
# Revenue Growth Rate = (Current Month Revenue - Previous Month Revenue) / Previous Month Revenue
revenue_growth_rate = monthly_total_revenue.pct_change()

# 4. Transaction Frequency (Monthly)
# Transaction Frequency = Total Orders / Total Customers
monthly_total_customers = df_order_details.groupby('year_month')['customer_id'].nunique()
transaction_frequency = monthly_total_orders / monthly_total_customers

# Output the results
print("Revenue per Customer (Monthly):")
print(monthly_revenue_per_customer_avg)
print("\nAverage Transaction Value (Monthly):")
print(average_transaction_value)
print("\nRevenue Growth Rate (Monthly):")
print(revenue_growth_rate)
print("\nTransaction Frequency (Monthly):")
print(transaction_frequency)

Revenue per Customer (Monthly):
year_month
2016-09     84.080000
2016-10    182.378025
2016-12     19.620000
2017-01    173.110050
2017-02    163.993264
2017-03    167.734377
2017-04    173.788698
2017-05    160.248330
2017-06    157.558206
2017-07    147.139324
2017-08    155.713766
2017-09    169.839545
2017-10    168.360587
2017-11    158.388494
2017-12    154.838971
2018-01    153.391688
2018-02    147.512387
2018-03    160.817102
2018-04    167.284260
2018-05    167.900793
2018-06    166.025701
2018-07    169.507430
2018-08    157.006345
2018-09    277.471250
2018-10    147.417500
Freq: M, Name: total_revenue, dtype: float64

Average Transaction Value (Monthly):
year_month
2016-09     84.080000
2016-10    182.378025
2016-12     19.620000
2017-01    173.110050
2017-02    163.993264
2017-03    167.734377
2017-04    173.788698
2017-05    160.248330
2017-06    157.558206
2017-07    147.139324
2017-08    155.713766
2017-09    169.839545
2017-10    168.360587
2017-11    158.388494
2017-